<a href="https://colab.research.google.com/github/minerva-mcgonagraph/chicken_and_doggo/blob/master/chicken_and_doggo_fastai_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Labradoodle or Fried Chicken?

![eight alternating pictures of similar-looking labradoodles and fried chicken](https://media.npr.org/assets/img/2016/03/11/labradoodle-or-fried-chicken_wide-14bb65d1f6416afcc4bdf20fbf1d4d6cbe381a5a.jpg?s=1400)

You've seen the meme. Now get ready for the AI that can tell the difference.

This notebook builds an artificial intelligence model that classifies an image as either a labradoodle (doggo) or fried chicken.

Hover over the \[  ] in each code block and click the little play button to run it.


---



First, we need to get the data set. I scraped some images off of Google and placed them neatly in a dropbox. Run the following code to grab them.

In [0]:
!wget --no-check-certificate \
    https://www.dropbox.com/s/4q6coicseieqinh/chicken_and_doggo_dataset.zip?dl=0 \
    -O /tmp/chicken_and_doggo_dataset.zip

Next we need to initialize a few things, import some libraries, and unzip the box of dogs and chicken. (Pause to consider those words out of context.)

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import os
import zipfile
from fastai.vision import *
from fastai.metrics import error_rate

local_zip = '/tmp/chicken_and_doggo_dataset.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()


Now we'll set the batch size and image size. The batch size is the number of examples used in each training iteration. The image size will be used to transform each image into a square of that many pixels on each side.

Geek out: The images in the data set are all different sizes. In convolutional neural networks with fully connected layers, like what's used here, the images must be transformed into the same size. This makes the model much faster but also means that images can get cropped and stretched and distorted. If you have nice, polite images where each image contains either a dog or some fried chicken it's not an issue. But tasks that involve object detection require a different approach.


In [0]:
batch_size = 32
image_size = 150

Now we'll retrieve the image classes.

This can be done manually since there are only 2 classes (doggo and chicken), but this code can scale to much larger data sets.

First we say where all the images are by defining PATH. Then we create a blank list of classes, and for each folder in the directory, add the name of the folder to the list, ignoring folders we don't want. Then we'll print the class names.

In [0]:
PATH = '/tmp/chicken_and_doggo_dataset'

classes = []
for d in os.listdir(PATH):
    if os.path.isdir(os.path.join(PATH, d)) and not d.startswith('.') and not d=='models':
        classes.append(d)
print("There are", len(classes), "classes:\n", classes)


Time to start the fun stuff. In this next cell we'll create the training and validation sets using some handy functions from the fastai library. Notice the image_size and batch_size from earlier!

We'll also look at the data, just to make sure it's what we want. If you run the cell over again, you will see different images. So if you want to be really sure the data is good and totally not just have an excuse to look at cute pictures of labradoodles and fried chicken, run the code as many times as you want.

In [0]:
data = ImageDataBunch.from_folder(
    PATH, 
    ds_tfms=get_transforms(),
    size=image_size,
    bs=batch_size,
    valid_pct=0.2).normalize(imagenet_stats)
print("There are", len(data.train_ds), "training images and",
    len(data.valid_ds), "validation images.")
data.show_batch(rows=3, figsize=(7,8))

And now we start the fancy stuff. The convolutional neural network (CNN) we're using here is resnet34. Residual Neural Networks were created by Kaiming He et al and won the ILSVRC in 2015, which is AI jargon for "they're really good." And thanks to the fastai library, we can build them with only a couple lines of code.

How ResNets and other CNNs work is a bit beyond the scope of this documentation. To give you a very general idea of how they work, a model is trained by taking a batch of images, extracting features and finding a best fit curve for how those features weigh in on determining the image class. This is repeated for a certain number of iterations (epochs) and the model adjusts the best fit curve. The amount of adjustment is based on a learning rate.

In this ResNet, the following code will run mock trainings using different learning rates. The final plot shown shows the loss at different learning rates.

In [0]:
learn = cnn_learner(data, models.resnet34, metrics=accuracy)
learn.lr_find();
learn.recorder.plot()

Now we'll pick two learning rates: 0.01 and 0.001 (in the graph, these are labeled as 1e-02 and 1e-03 respectively). There's reasons for picking these two but that's beyond the scope of this documentation. Then train the model, cycling between these two learning rates.

In [0]:
learn.fit_one_cycle(4, max_lr=slice(1e-3,1e-2))

So what are we looking at? Where are the pictures of labradoodles and fried chicken?

Well, the table shows the accuracy of the model. It's a probability in the form of a decimal, so an accuracy in the final epoch of 0.98 means the model can accurately determine if the image is of a labradoodle or fried chicken correctly 98% of the time. The accuracy numbers you get may be slightly different (if they look weird, just run it again) due to the randomness of training.

But we just built a high accuracy classification model with only a few lines of code! And for those who know a little about data science, we did this without any effort in tuning parameters!

One last thing! Run the following code to free up memory (it will say it has crashed and is restarting). Then enjoy some more pictures of labradoodles and fried chicken.

In [0]:
import os, signal
os.kill(os.getpid(), signal.SIGKILL)

![eight more alternating images of similar-looking labradoodles and fried chicken](https://www.dropbox.com/s/tphoccpoqa33wmq/doggochicken2.jpg?dl=0)